# Building LangChain Agents to Automate Tasks in Python

## Introduction

## Setup

```bash
$ conda create -n langchain python=3.9 -y
$ conda activate langchain
$ pip install langchain langchain_openai langchain_community ipykernel python-dotenv
$ ipython kernel install --user --name=langchain
```

```bash
$ touch .env 
$ vim .env  # Paste your OPENAI key
```

```python
from dotenv import load_dotenv()
```

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

In [12]:
from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=api_key)

question = "Is Messi the best footballer of all time?"
output = llm.invoke(question)

print(output)



Many people consider Messi to be one of the best footballers of all time, if not the best. His incredible skills, record-breaking achievements, and consistent performance at the highest level have cemented his place among the greatest footballers in history. However, opinions on the best footballer of all time may vary and some may argue that other players, such as Pele or Diego Maradona, also deserve the title. Ultimately, it is a matter of personal preference.


### Prompting templates

In [16]:
from langchain_core.prompts import PromptTemplate

template = "You are a dreamy poet. Compose a poem on {subject}."

prompt_template = PromptTemplate(template=template, input_variables=['subject'])

prompt_template.invoke({"subject": "Qur'an"})

StringPromptValue(text="You are a dreamy poet. Compose a poem on Qur'an.")

In [21]:
llm_chain = prompt_template | llm

llm_chain.invoke({'subject': "Qur'an"})

"\n\nThe Qur'an, a divine book of guidance\nSent down from above, a heavenly fragrance\nRevealing wisdom, truth, and light\nA source of solace, in the darkest of night\n\nIn its verses, lies a message profound\nA roadmap to salvation, a treasure found\nIn every word, there's an ocean of meaning\nA fountain of knowledge, forever streaming\n\nIts beauty transcends the boundaries of time\nA masterpiece of prose, a symphony sublime\nEach verse, a melody of love and mercy\nA soothing balm for the hearts that are thirsty\n\nGuiding us towards the path of righteousness\nTeaching us to strive for goodness and bliss\nWith every verse, our faith is renewed\nAs we delve deeper, our souls are imbued\n\nThe Qur'an, a source of comfort and hope\nA lifeline for those who struggle and cope\nIn its pages, we find solace and peace\nAs our worries and fears, it begins to ease\n\nIts teachings, a light that illuminates our way\nAs we navigate through life, day by day\nWith every recitation, our hearts are

### Chat models

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a master procrastinator."),
        ("human", "Why do you do this?"),
        ("ai", "I have no idea"),
        ("human", "Respond to this question: {question}")
    ]
)


In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=api_key)

llm_chain = prompt_template | llm
question = "Why do you procrastinate?"

response = llm_chain.invoke({'question': question})


In [29]:
response.content

'I procrastinate because I find it challenging to focus on tasks until the last minute.'

### Managing chat model memory

In [34]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(openai_api_key=api_key)

history = ChatMessageHistory()
history.add_ai_message('Hi, ask me anything on football.')
history.add_user_message('Describe a metaphor for football that goes beyond human thought.')

response = llm.invoke(history.messages)

In [42]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=ChatOpenAI(openai_api_key=api_key))

summary_chain = ConversationChain(llm=llm, memory=memory, verbose=True)

### More on chains

In [43]:
from langchain_core.output_parsers.string import StrOutputParser

destination_prompt = PromptTemplate(
    input_variables=['destination'],
    template='I am planning a trip to {destination}. Can you suggest some activities to do there?'
)

activities_prompt = PromptTemplate(
    input_variables=['activities'],
    template='I only have one day, so you can create an itinerary from your top three activities: {activities}'
)

seq_chain = ({"activities": destination_prompt | llm | StrOutputParser()} 
                           | activities_prompt | llm | StrOutputParser()) 

In [44]:
seq_chain.invoke({'destination': 'tashkent'})

'\n\nPlease note that it may not be possible to do all of these activities in one day, so I recommend prioritizing the ones that interest you the most. Enjoy your time in Tashkent!'

There is an error in this crap. Fix it tomorrow, bex

## Conclusion